In [1]:
import requests
import pandas as pd
from datetime import datetime

def scrape_fpl_ownership_simple():
    """
    Scrape FPL ownership data - simple format with just:
    - web_name
    - gameweek  
    - selected_by_percent
    """
    
    # FPL API endpoint
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    
    print("Fetching FPL ownership data...")
    
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Error: API returned status code {response.status_code}")
        return None
    
    data = response.json()
    
    # Get current gameweek
    current_gameweek = None
    for event in data['events']:
        if event['is_current']:
            current_gameweek = event['id']
            break
    
    if current_gameweek is None:
        print("Warning: No current gameweek found, using last completed GW")
        for event in data['events']:
            if event['finished']:
                current_gameweek = event['id']
    
    print(f"\nCurrent Gameweek: {current_gameweek}")
    print(f"Total players: {len(data['elements'])}")
    
    # Extract simple ownership data
    ownership_data = []
    
    for player in data['elements']:
        ownership_data.append({
            'web_name': player['web_name'],
            'gameweek': current_gameweek,
            'selected_by_percent': float(player['selected_by_percent'])
        })
    
    # Create DataFrame
    df = pd.DataFrame(ownership_data)
    
    # Sort by ownership (highest first)
    df = df.sort_values('selected_by_percent', ascending=False).reset_index(drop=True)
    
    return df, current_gameweek

if __name__ == "__main__":
    # Scrape the data
    df, gameweek = scrape_fpl_ownership_simple()
    
    if df is not None:
        # Save to CSV
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"fpl_ownership_gw{gameweek}_{timestamp}.csv"
        
        df.to_csv(output_file, index=False)
        
        print(f"\n{'='*60}")
        print(f"✓ SUCCESS!")
        print(f"{'='*60}")
        print(f"Saved {len(df)} players to: {output_file}")
        print(f"\nTop 10 Most Owned Players:")
        print(f"{'-'*60}")
        
        for idx, row in df.head(10).iterrows():
            print(f"{idx+1:2d}. {row['web_name']:20s} | GW{row['gameweek']} | {row['selected_by_percent']:5.1f}%")
        
        print(f"\nBottom 10 Least Owned Players:")
        print(f"{'-'*60}")
        
        for idx, row in df.tail(10).iterrows():
            print(f"{idx+1:3d}. {row['web_name']:20s} | GW{row['gameweek']} | {row['selected_by_percent']:5.1f}%")
        
        print(f"\n{'='*60}")
        print(f"CSV Format Preview:")
        print(f"{'='*60}")
        print(df.head(5).to_string(index=False))
        
        print(f"\n✓ File ready: {output_file}")

Fetching FPL ownership data...

Current Gameweek: 10
Total players: 748

✓ SUCCESS!
Saved 748 players to: fpl_ownership_gw10_20251105_013623.csv

Top 10 Most Owned Players:
------------------------------------------------------------
 1. Haaland              | GW10 |  69.8%
 2. Semenyo              | GW10 |  64.4%
 3. João Pedro           | GW10 |  45.9%
 4. Gabriel              | GW10 |  41.5%
 5. Dúbravka             | GW10 |  34.4%
 6. Guéhi                | GW10 |  34.4%
 7. Van de Ven           | GW10 |  32.7%
 8. Raya                 | GW10 |  31.6%
 9. Kudus                | GW10 |  29.5%
10. Mbeumo               | GW10 |  28.8%

Bottom 10 Least Owned Players:
------------------------------------------------------------
739. L.Guilherme          | GW10 |   0.0%
740. Earthy               | GW10 |   0.0%
741. M.Fernandes          | GW10 |   0.0%
742. S.Magassa            | GW10 |   0.0%
743. Fabianski            | GW10 |   0.0%
744. Mayers               | GW10 |   0.0%
745. Fer Ló